# Notebook del Hecho Servicio - Proceso ETL

## 1. Configuración de conexiones (fuente y bodega)

In [3]:
import yaml
from sqlalchemy import create_engine
import pandas as pd

# Leer configuración desde archivo externo
with open('config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_fuente = config['fuente']
    config_bodega = config['bodega']

# URLs de conexión
url_fuente = (f"{config_fuente['driver']}://{config_fuente['user']}:{config_fuente['password']}@"
              f"{config_fuente['host']}:{config_fuente['port']}/{config_fuente['db']}")

url_bodega = (f"{config_bodega['driver']}://{config_bodega['user']}:{config_bodega['password']}@"
              f"{config_bodega['host']}:{config_bodega['port']}/{config_bodega['db']}")

# Engines SQLAlchemy
engine_fuente = create_engine(url_fuente)
engine_bodega = create_engine(url_bodega)

## 2 y 3. Extracción y Transformación de datos para la tabla de hechos - Extract and Transform

In [4]:
# Extraer datos de servicios activos
query_servicios = """
SELECT 
    id AS id_servicio,
    cliente_id,
    ciudad_origen_id,
    ciudad_destino_id,
    mensajero_id,
    fecha_solicitud,
    hora_solicitud,
    tipo_servicio_id,
    tipo_vehiculo_id,
    activo,
    prioridad
FROM mensajeria_servicio
WHERE activo = TRUE;
"""

df_servicios = pd.read_sql(query_servicios, engine_fuente)

In [5]:
df_servicios.head()

,id_servicio,cliente_id,ciudad_origen_id,ciudad_destino_id,mensajero_id,fecha_solicitud,hora_solicitud,tipo_servicio_id,tipo_vehiculo_id,activo,prioridad
0,35,5,1,1,7.0,2023-10-26,11:18:14,3,2,True,Media: De 1 a 3 horas
1,46,4,1,1,12.0,2023-11-09,18:11:22,3,2,True,Media: De 1 a 3 horas
2,45,5,1,1,12.0,2023-11-09,17:54:40,3,2,True,Media: De 1 a 3 horas
3,29,5,1,1,7.0,2023-10-13,17:13:13,3,3,True,Alta: En una hora
4,8,5,1,1,7.0,2023-09-19,16:30:05,1,3,True,Alta: En una hora


In [6]:
# Extraer estados del servicio
query_estados_servicio = """
SELECT 
    es.servicio_id,
    es.estado_id,
    e.nombre AS nombre_estado,
    es.fecha,
    es.hora
FROM mensajeria_estadosservicio es
JOIN mensajeria_estado e ON es.estado_id = e.id
ORDER BY es.servicio_id, es.fecha, es.hora;
"""

df_estados = pd.read_sql(query_estados_servicio, engine_fuente)

In [7]:
df_estados["hora"].unique()[:5]

array([datetime.time(16, 22, 18), datetime.time(17, 51, 20),
       datetime.time(12, 2, 48, 844000), datetime.time(12, 16),
       datetime.time(17, 7, 55)], dtype=object)

In [8]:
# Unificación de fecha y hora en un timestamp
df_estados["hora_str"] = df_estados["hora"].apply(lambda x: x.strftime("%H:%M:%S"))

In [9]:
df_estados["timestamp_estado"] = pd.to_datetime(
    df_estados["fecha"].astype(str) + " " + df_estados["hora_str"],
    format="%Y-%m-%d %H:%M:%S"
)

In [10]:
df_estados[["fecha", "hora", "hora_str", "timestamp_estado"]].head(10)

,fecha,hora,hora_str,timestamp_estado
0,2023-09-19,16:22:18,16:22:18,2023-09-19 16:22:18
1,2023-10-13,17:51:20,17:51:20,2023-10-13 17:51:20
2,2023-10-31,12:02:48.844000,12:02:48,2023-10-31 12:02:48
3,2023-10-31,12:16:00,12:16:00,2023-10-31 12:16:00
4,2023-10-31,17:07:55,17:07:55,2023-10-31 17:07:55
5,2023-09-19,16:30:05,16:30:05,2023-09-19 16:30:05
6,2023-12-20,20:14:43,20:14:43,2023-12-20 20:14:43
7,2024-02-14,15:34:18,15:34:18,2024-02-14 15:34:18
8,2024-04-09,16:08:35,16:08:35,2024-04-09 16:08:35
9,2023-09-19,16:30:05,16:30:05,2023-09-19 16:30:05


In [11]:
df_estados.dtypes[["hora_str", "timestamp_estado"]]

hora_str                    object
timestamp_estado    datetime64[ns]
dtype: object

In [12]:
df_estados.sort_values(["servicio_id", "timestamp_estado"]).head(10)

,servicio_id,estado_id,nombre_estado,fecha,hora,hora_str,timestamp_estado
0,7,1,Iniciado,2023-09-19,16:22:18,16:22:18,2023-09-19 16:22:18
1,7,2,Con mensajero Asignado,2023-10-13,17:51:20,17:51:20,2023-10-13 17:51:20
2,7,4,Recogido por mensajero,2023-10-31,12:02:48.844000,12:02:48,2023-10-31 12:02:48
3,7,6,Terminado completo,2023-10-31,12:16:00,12:16:00,2023-10-31 12:16:00
4,7,5,Entregado en destino,2023-10-31,17:07:55,17:07:55,2023-10-31 17:07:55
5,8,1,Iniciado,2023-09-19,16:30:05,16:30:05,2023-09-19 16:30:05
6,8,2,Con mensajero Asignado,2023-12-20,20:14:43,20:14:43,2023-12-20 20:14:43
7,8,4,Recogido por mensajero,2024-02-14,15:34:18,15:34:18,2024-02-14 15:34:18
8,8,5,Entregado en destino,2024-04-09,16:08:35,16:08:35,2024-04-09 16:08:35
9,9,1,Iniciado,2023-09-19,16:30:05,16:30:05,2023-09-19 16:30:05


## Crear pivot con los estados importantes

In [13]:
df_pivot = df_estados[["servicio_id", "nombre_estado", "timestamp_estado"]]

In [14]:
df_pivot = df_pivot.pivot_table(
    index="servicio_id",
    columns="nombre_estado",
    values="timestamp_estado",
    aggfunc="min"
).reset_index()

In [15]:
df_pivot.head()

nombre_estado,servicio_id,Con mensajero Asignado,Con novedad,Entregado en destino,Iniciado,Recogido por mensajero,Terminado completo
0,7,2023-10-13 17:51:20,NaT,2023-10-31 17:07:55,2023-09-19 16:22:18,2023-10-31 12:02:48,2023-10-31 12:16:00
1,8,2023-12-20 20:14:43,NaT,2024-04-09 16:08:35,2023-09-19 16:30:05,2024-02-14 15:34:18,NaT
2,9,2023-12-28 19:33:01,NaT,NaT,2023-09-19 16:30:05,NaT,NaT
3,10,2023-12-28 19:33:07,NaT,2024-03-10 09:58:27,2023-09-19 16:35:52,2024-02-18 00:21:47,NaT
4,11,2023-12-09 13:13:59,NaT,NaT,2023-09-19 16:37:54,2024-01-31 10:29:55,NaT


In [16]:
estados_interes = [
    "Iniciado",
    "Con mensajero Asignado",
    "Recogido por mensajero",
    "Entregado en destino",
    "Terminado completo"
]

In [17]:
# Filtramos solo esos
df_filtrado = df_estados[df_estados["nombre_estado"].isin(estados_interes)]

In [18]:
# Pivot con solo los estados que nos interesan
df_pivot = df_filtrado.pivot_table(
    index="servicio_id",
    columns="nombre_estado",
    values="timestamp_estado",
    aggfunc="min"
).reset_index()

In [19]:
df_pivot.head()

nombre_estado,servicio_id,Con mensajero Asignado,Entregado en destino,Iniciado,Recogido por mensajero,Terminado completo
0,7,2023-10-13 17:51:20,2023-10-31 17:07:55,2023-09-19 16:22:18,2023-10-31 12:02:48,2023-10-31 12:16:00
1,8,2023-12-20 20:14:43,2024-04-09 16:08:35,2023-09-19 16:30:05,2024-02-14 15:34:18,NaT
2,9,2023-12-28 19:33:01,NaT,2023-09-19 16:30:05,NaT,NaT
3,10,2023-12-28 19:33:07,2024-03-10 09:58:27,2023-09-19 16:35:52,2024-02-18 00:21:47,NaT
4,11,2023-12-09 13:13:59,NaT,2023-09-19 16:37:54,2024-01-31 10:29:55,NaT


In [20]:
# Renombrar columnas
df_pivot = df_pivot.rename(columns={
    "Iniciado": "fecha_inicio",
    "Con mensajero Asignado": "fecha_asignado",
    "Recogido por mensajero": "fecha_recogido",
    "Entregado en destino": "fecha_entregado",
    "Terminado completo": "fecha_cerrado"
})

In [21]:
# Calcular diferencias de tiempo
df_pivot["tiempo_total_servicio_horas"] = (
    df_pivot["fecha_cerrado"] - df_pivot["fecha_inicio"]
).dt.total_seconds() / 3600

df_pivot["tiempo_estado_iniciado_horas"] = (
    df_pivot["fecha_asignado"] - df_pivot["fecha_inicio"]
).dt.total_seconds() / 3600

df_pivot["tiempo_estado_asignado_horas"] = (
    df_pivot["fecha_recogido"] - df_pivot["fecha_asignado"]
).dt.total_seconds() / 3600

df_pivot["tiempo_estado_recogido_horas"] = (
    df_pivot["fecha_entregado"] - df_pivot["fecha_recogido"]
).dt.total_seconds() / 3600

df_pivot["tiempo_estado_entregado_horas"] = (
    df_pivot["fecha_cerrado"] - df_pivot["fecha_entregado"]
).dt.total_seconds() / 3600

In [22]:
df_pivot[[
    "servicio_id", 
    "fecha_inicio", "fecha_asignado", "fecha_recogido", 
    "fecha_entregado", "fecha_cerrado",
    "tiempo_total_servicio_horas", 
    "tiempo_estado_iniciado_horas",
    "tiempo_estado_asignado_horas",
    "tiempo_estado_recogido_horas",
    "tiempo_estado_entregado_horas"
]].head(10)

nombre_estado,servicio_id,fecha_inicio,fecha_asignado,fecha_recogido,fecha_entregado,fecha_cerrado,tiempo_total_servicio_horas,tiempo_estado_iniciado_horas,tiempo_estado_asignado_horas,tiempo_estado_recogido_horas,tiempo_estado_entregado_horas
0,7,2023-09-19 16:22:18,2023-10-13 17:51:20,2023-10-31 12:02:48,2023-10-31 17:07:55,2023-10-31 12:16:00,1003.895000,577.483889,426.191111,5.085278,-4.865278
1,8,2023-09-19 16:30:05,2023-12-20 20:14:43,2024-02-14 15:34:18,2024-04-09 16:08:35,NaT,NaN,2211.743889,1339.326389,1320.571389,NaN
2,9,2023-09-19 16:30:05,2023-12-28 19:33:01,NaT,NaT,NaT,NaN,2403.048889,NaN,NaN,NaN
3,10,2023-09-19 16:35:52,2023-12-28 19:33:07,2024-02-18 00:21:47,2024-03-10 09:58:27,NaT,NaN,2402.954167,1228.811111,513.611111,NaN
4,11,2023-09-19 16:37:54,2023-12-09 13:13:59,2024-01-31 10:29:55,NaT,NaT,NaN,1940.601389,1269.265556,NaN,NaN
5,12,2023-09-19 16:49:17,2023-10-07 13:47:46,2023-10-29 00:41:36,2023-10-29 00:44:22,2023-10-29 00:46:00,943.945278,428.974722,514.897222,0.046111,0.027222
6,13,2023-09-19 16:52:14,2023-10-07 13:08:30,2023-10-13 23:55:07,2024-01-02 03:32:02,2024-01-02 03:32:31,2506.671389,428.271111,154.776944,1923.615278,0.008056
7,14,2023-09-19 16:59:50,2023-10-07 13:07:23,2024-02-22 16:00:36,2024-02-23 12:09:52,2024-03-26 16:22:36,4535.379444,428.125833,3314.886944,20.154444,772.212222
8,15,2023-09-19 17:03:49,2023-10-07 12:53:33,NaT,NaT,NaT,NaN,427.828889,NaN,NaN,NaN
9,16,2023-09-19 17:07:28,2023-10-07 12:52:03,2024-01-02 03:36:10,2024-01-02 03:36:37,2024-01-02 03:36:56,2506.491111,427.743056,2078.735278,0.007500,0.005278


In [23]:
# Cuantos estados ha tenido cada servicio
cantidad_estados = df_estados.groupby("servicio_id").size().reset_index(name="cantidad_estados")
cantidad_estados

,servicio_id,cantidad_estados
0,7,5
1,8,4
2,9,2
3,10,4
4,11,3
...,...,...
28425,28464,6
28426,28465,4
28427,28466,5
28428,28467,5


In [24]:
# Unir con el pivot
df_pivot = df_pivot.merge(cantidad_estados, on="servicio_id", how="left")

In [25]:
df_novedades = pd.read_sql("SELECT * FROM mensajeria_novedadesservicio", engine_fuente)

In [26]:
# Contar novedades por servicio
cantidad_novedades = df_novedades.groupby("servicio_id").size().reset_index(name="cantidad_novedades")
cantidad_novedades

,servicio_id,cantidad_novedades
0,29,2
1,51,8
2,57,3
3,68,3
4,73,1
...,...,...
3270,28455,1
3271,28461,1
3272,28464,1
3273,28466,1


In [27]:
df_novedades.columns

Index(['id', 'fecha_novedad', 'tipo_novedad_id', 'descripcion', 'servicio_id',
       'es_prueba', 'mensajero_id'],
      dtype='object')

In [28]:
df_pivot = df_pivot.merge(cantidad_novedades, on="servicio_id", how="left")
df_pivot["cantidad_novedades"] = df_pivot["cantidad_novedades"].fillna(0).astype(int)

In [29]:
df_pivot[[
    "servicio_id", 
    "cantidad_estados",
    "cantidad_novedades"
]].head(10)

,servicio_id,cantidad_estados,cantidad_novedades
0,7,5,0
1,8,4,0
2,9,2,0
3,10,4,0
4,11,3,0
5,12,6,0
6,13,5,0
7,14,5,0
8,15,2,0
9,16,5,0


In [30]:
# Tiempos entre etapas (en minutos)
df_pivot["tiempo_estado_iniciado"] = (df_pivot["fecha_asignado"] - df_pivot["fecha_inicio"]).dt.total_seconds() / 60
df_pivot["tiempo_estado_asignado"] = (df_pivot["fecha_recogido"] - df_pivot["fecha_asignado"]).dt.total_seconds() / 60
df_pivot["tiempo_estado_recogido"] = (df_pivot["fecha_entregado"] - df_pivot["fecha_recogido"]).dt.total_seconds() / 60
df_pivot["tiempo_estado_entregado"] = (df_pivot["fecha_cerrado"] - df_pivot["fecha_entregado"]).dt.total_seconds() / 60

# Tiempo total del servicio
df_pivot["tiempo_total_servicio"] = (df_pivot["fecha_cerrado"] - df_pivot["fecha_inicio"]).dt.total_seconds() / 60

In [31]:
# Redondear y rellenar con 0 los NaN
columnas_tiempo = [
    "tiempo_estado_iniciado", "tiempo_estado_asignado",
    "tiempo_estado_recogido", "tiempo_estado_entregado",
    "tiempo_total_servicio"
]

df_pivot[columnas_tiempo] = df_pivot[columnas_tiempo].fillna(0).round(2)

In [32]:
df_pivot.head(3)

,servicio_id,fecha_asignado,fecha_entregado,fecha_inicio,fecha_recogido,fecha_cerrado,tiempo_total_servicio_horas,tiempo_estado_iniciado_horas,tiempo_estado_asignado_horas,tiempo_estado_recogido_horas,tiempo_estado_entregado_horas,cantidad_estados,cantidad_novedades,tiempo_estado_iniciado,tiempo_estado_asignado,tiempo_estado_recogido,tiempo_estado_entregado,tiempo_total_servicio
0,7,2023-10-13 17:51:20,2023-10-31 17:07:55,2023-09-19 16:22:18,2023-10-31 12:02:48,2023-10-31 12:16:00,1003.895,577.483889,426.191111,5.085278,-4.865278,5,0,34649.03,25571.47,305.12,-291.92,60233.7
1,8,2023-12-20 20:14:43,2024-04-09 16:08:35,2023-09-19 16:30:05,2024-02-14 15:34:18,NaT,NaN,2211.743889,1339.326389,1320.571389,NaN,4,0,132704.63,80359.58,79234.28,0.00,0.0
2,9,2023-12-28 19:33:01,NaT,2023-09-19 16:30:05,NaT,NaT,NaN,2403.048889,NaN,NaN,NaN,2,0,144182.93,0.00,0.00,0.00,0.0


In [33]:
query = """
SELECT 
    id AS servicio_id,
    cliente_id,
    mensajero_id,
    origen_id AS sede_id,
    fecha_solicitud AS fecha,
    hora_solicitud AS hora
FROM mensajeria_servicio
"""

df_servicios = pd.read_sql(query, engine_fuente)
df_servicios

,servicio_id,cliente_id,mensajero_id,sede_id,fecha,hora
0,34,5,NaN,241,2023-10-26,09:46:03
1,35,5,7.0,236,2023-10-26,11:18:14
2,36,5,NaN,236,2023-10-28,19:21:01
3,41,5,NaN,241,2023-11-07,09:46:09
4,42,5,NaN,241,2023-11-07,09:46:10
...,...,...,...,...,...,...
28425,28267,11,8.0,28,2024-08-30,10:52:02
28426,28363,11,22.0,442,2024-08-30,16:02:24
28427,28403,5,30.0,243,2024-08-31,07:52:50
28428,28438,11,39.0,47,2024-08-31,10:42:03


In [34]:
# Unir df_pivot con los datos originales de servicios
df_hecho = df_pivot.merge(
    df_servicios[["servicio_id", "cliente_id", "mensajero_id", "sede_id", "fecha", "hora"]],
    on="servicio_id",
    how="left"
)

In [35]:
# Cargar dimensiones
dim_fecha = pd.read_csv("dim_fecha.csv")
dim_tiempo = pd.read_csv("dim_tiempo.csv")
df_hecho

,servicio_id,fecha_asignado,fecha_entregado,fecha_inicio,fecha_recogido,fecha_cerrado,tiempo_total_servicio_horas,tiempo_estado_iniciado_horas,tiempo_estado_asignado_horas,tiempo_estado_recogido_horas,...,tiempo_estado_iniciado,tiempo_estado_asignado,tiempo_estado_recogido,tiempo_estado_entregado,tiempo_total_servicio,cliente_id,mensajero_id,sede_id,fecha,hora
0,7,2023-10-13 17:51:20,2023-10-31 17:07:55,2023-09-19 16:22:18,2023-10-31 12:02:48,2023-10-31 12:16:00,1003.895,577.483889,426.191111,5.085278,...,34649.03,25571.47,305.12,-291.92,60233.7,5,1.0,177,2023-09-19,16:22:18
1,8,2023-12-20 20:14:43,2024-04-09 16:08:35,2023-09-19 16:30:05,2024-02-14 15:34:18,NaT,NaN,2211.743889,1339.326389,1320.571389,...,132704.63,80359.58,79234.28,0.00,0.0,5,7.0,15,2023-09-19,16:30:05
2,9,2023-12-28 19:33:01,NaT,2023-09-19 16:30:05,NaT,NaT,NaN,2403.048889,NaN,NaN,...,144182.93,0.00,0.00,0.00,0.0,5,7.0,15,2023-09-19,16:30:05
3,10,2023-12-28 19:33:07,2024-03-10 09:58:27,2023-09-19 16:35:52,2024-02-18 00:21:47,NaT,NaN,2402.954167,1228.811111,513.611111,...,144177.25,73728.67,30816.67,0.00,0.0,5,7.0,15,2023-09-19,16:35:52
4,11,2023-12-09 13:13:59,NaT,2023-09-19 16:37:54,2024-01-31 10:29:55,NaT,NaN,1940.601389,1269.265556,NaN,...,116436.08,76155.93,0.00,0.00,0.0,5,7.0,15,2023-09-19,16:37:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28425,28464,2024-08-31 13:12:56,2024-08-31 13:57:35,2024-08-31 12:48:13,2024-08-31 13:38:22,NaT,NaN,0.411944,0.423889,0.320278,...,24.72,25.43,19.22,0.00,0.0,9,25.0,381,2024-08-31,12:48:13
28426,28465,2024-08-31 13:41:25,2024-08-31 15:22:35,2024-08-31 13:31:15,2024-08-31 14:33:49,NaT,NaN,0.169444,0.873333,0.812778,...,10.17,52.40,48.77,0.00,0.0,3,32.0,11,2024-08-31,13:31:15
28427,28466,2024-08-31 14:11:00,NaT,2024-08-31 14:03:45,2024-08-31 15:04:26,NaT,NaN,0.120833,0.890556,NaN,...,7.25,53.43,0.00,0.00,0.0,25,25.0,384,2024-08-31,14:03:45
28428,28467,2024-08-31 14:29:10,NaT,2024-08-31 14:12:17,NaT,NaT,NaN,0.281389,NaN,NaN,...,16.88,0.00,0.00,0.00,0.0,25,25.0,383,2024-08-31,14:12:17


In [36]:
# Extraemos fecha y hora
df_hecho["fecha_inicio_servicio"] = pd.to_datetime(df_hecho["fecha"])
df_hecho["hora_inicio_servicio"] = pd.to_datetime(
    df_hecho["hora"].astype(str), 
    format='%H:%M:%S'
).dt.time

In [37]:
df_hecho = df_hecho.merge(
    dim_fecha,
    left_on=[
        df_hecho["fecha_inicio_servicio"].dt.year,
        df_hecho["fecha_inicio_servicio"].dt.month,
        df_hecho["fecha_inicio_servicio"].dt.day
    ],
    right_on=["anio", "mes", "dia"],
    how="left"
)

df_hecho = df_hecho.rename(columns={"id_fecha": "id_fecha_inicio"})

In [38]:
df_hecho = df_hecho.merge(
    dim_tiempo,
    left_on="hora_inicio_servicio",
    right_on="hora",
    how="left"
)

df_hecho = df_hecho.rename(columns={"id_tiempo": "id_tiempo_inicio"})

In [39]:
# DataFrame final limpio para la tabla de hechos servicio
df_hecho_servicio = df_hecho[[
    "servicio_id",
    "cliente_id",
    "mensajero_id",
    "sede_id",
    "id_fecha_inicio",
    "tiempo_estado_iniciado",
    "tiempo_estado_asignado",
    "tiempo_estado_recogido",
    "tiempo_estado_entregado",
    "tiempo_total_servicio",
    "cantidad_estados",
    "cantidad_novedades",
    "fecha_inicio",
    "fecha_asignado",
    "fecha_recogido",
    "fecha_entregado",
    "fecha_cerrado"
]].copy()

# Agregar hora de inicio para el analisis por hora
df_hecho_servicio["hora_inicio"] = df_hecho["hora_inicio_servicio"]

In [40]:
print(df_hecho.columns.tolist())

['servicio_id', 'fecha_asignado', 'fecha_entregado', 'fecha_inicio', 'fecha_recogido', 'fecha_cerrado', 'tiempo_total_servicio_horas', 'tiempo_estado_iniciado_horas', 'tiempo_estado_asignado_horas', 'tiempo_estado_recogido_horas', 'tiempo_estado_entregado_horas', 'cantidad_estados', 'cantidad_novedades', 'tiempo_estado_iniciado', 'tiempo_estado_asignado', 'tiempo_estado_recogido', 'tiempo_estado_entregado', 'tiempo_total_servicio', 'cliente_id', 'mensajero_id', 'sede_id', 'fecha', 'hora_x', 'fecha_inicio_servicio', 'hora_inicio_servicio', 'id_fecha_inicio', 'anio', 'date', 'anio_mes', 'mes', 'mes_str', 'dia', 'dia_str', 'trimestre', 'semana_anio', 'fecha_carga_x', 'hora_y', 'minuto', 'segundo', 'hora_completa', 'momento_dia', 'fecha_carga_y']


In [41]:
print(df_hecho_servicio.columns.tolist())

['servicio_id', 'cliente_id', 'mensajero_id', 'sede_id', 'id_fecha_inicio', 'tiempo_estado_iniciado', 'tiempo_estado_asignado', 'tiempo_estado_recogido', 'tiempo_estado_entregado', 'tiempo_total_servicio', 'cantidad_estados', 'cantidad_novedades', 'fecha_inicio', 'fecha_asignado', 'fecha_recogido', 'fecha_entregado', 'fecha_cerrado', 'hora_inicio']


In [42]:
df_hecho_servicio

,servicio_id,cliente_id,mensajero_id,sede_id,id_fecha_inicio,tiempo_estado_iniciado,tiempo_estado_asignado,tiempo_estado_recogido,tiempo_estado_entregado,tiempo_total_servicio,cantidad_estados,cantidad_novedades,fecha_inicio,fecha_asignado,fecha_recogido,fecha_entregado,fecha_cerrado,hora_inicio
0,7,5,1.0,177,111,34649.03,25571.47,305.12,-291.92,60233.7,5,0,2023-09-19 16:22:18,2023-10-13 17:51:20,2023-10-31 12:02:48,2023-10-31 17:07:55,2023-10-31 12:16:00,16:22:18
1,8,5,7.0,15,111,132704.63,80359.58,79234.28,0.00,0.0,4,0,2023-09-19 16:30:05,2023-12-20 20:14:43,2024-02-14 15:34:18,2024-04-09 16:08:35,NaT,16:30:05
2,9,5,7.0,15,111,144182.93,0.00,0.00,0.00,0.0,2,0,2023-09-19 16:30:05,2023-12-28 19:33:01,NaT,NaT,NaT,16:30:05
3,10,5,7.0,15,111,144177.25,73728.67,30816.67,0.00,0.0,4,0,2023-09-19 16:35:52,2023-12-28 19:33:07,2024-02-18 00:21:47,2024-03-10 09:58:27,NaT,16:35:52
4,11,5,7.0,15,111,116436.08,76155.93,0.00,0.00,0.0,3,0,2023-09-19 16:37:54,2023-12-09 13:13:59,2024-01-31 10:29:55,NaT,NaT,16:37:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28425,28464,9,25.0,381,458,24.72,25.43,19.22,0.00,0.0,6,1,2024-08-31 12:48:13,2024-08-31 13:12:56,2024-08-31 13:38:22,2024-08-31 13:57:35,NaT,12:48:13
28426,28465,3,32.0,11,458,10.17,52.40,48.77,0.00,0.0,4,0,2024-08-31 13:31:15,2024-08-31 13:41:25,2024-08-31 14:33:49,2024-08-31 15:22:35,NaT,13:31:15
28427,28466,25,25.0,384,458,7.25,53.43,0.00,0.00,0.0,5,1,2024-08-31 14:03:45,2024-08-31 14:11:00,2024-08-31 15:04:26,NaT,NaT,14:03:45
28428,28467,25,25.0,383,458,16.88,0.00,0.00,0.00,0.0,5,2,2024-08-31 14:12:17,2024-08-31 14:29:10,NaT,NaT,NaT,14:12:17


## 4. Guardado de la dimensión - Load

In [43]:
df_hecho_servicio.columns

Index(['servicio_id', 'cliente_id', 'mensajero_id', 'sede_id',
       'id_fecha_inicio', 'tiempo_estado_iniciado', 'tiempo_estado_asignado',
       'tiempo_estado_recogido', 'tiempo_estado_entregado',
       'tiempo_total_servicio', 'cantidad_estados', 'cantidad_novedades',
       'fecha_inicio', 'fecha_asignado', 'fecha_recogido', 'fecha_entregado',
       'fecha_cerrado', 'hora_inicio'],
      dtype='object')

In [44]:
# Guardar como CSV
df_hecho_servicio.to_csv("hecho_servicio.csv", index=False)

In [45]:
# Cargar en la bodega de datos OLAP
df_hecho_servicio.to_sql("hecho_servicio", engine_bodega, if_exists="replace", index=False)

430

In [46]:
# Verificación del archivo
df_verificacion = pd.read_csv("hecho_servicio.csv")
print(df_verificacion.head())

   servicio_id  cliente_id  mensajero_id  sede_id  id_fecha_inicio  \
0            7           5           1.0      177              111   
1            8           5           7.0       15              111   
2            9           5           7.0       15              111   
3           10           5           7.0       15              111   
4           11           5           7.0       15              111   

   tiempo_estado_iniciado  tiempo_estado_asignado  tiempo_estado_recogido  \
0                34649.03                25571.47                  305.12   
1               132704.63                80359.58                79234.28   
2               144182.93                    0.00                    0.00   
3               144177.25                73728.67                30816.67   
4               116436.08                76155.93                    0.00   

   tiempo_estado_entregado  tiempo_total_servicio  cantidad_estados  \
0                  -291.92                602